In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

Creating the soup

In [37]:
url = "https://www.craftbeer.com/styles/american-amber-lager"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

Scrapping the website leaving 3 sec between page to scrap. I am getting the ABV, SRM and IBU

In [ ]:
beers_to_scrape2 = ['Amber-Ale','American-Amber-Lager','American-Barley-Wine','American-Black-Ale','American-Brett','American-Brown-Ale','American-Cream-Ale',
                    'American-Imperial-Porter','American-Imperial-Red-Ale','American-Imperial-Stout','american-india-pale-ale','American-Lager','American-Pale-Ale','American-Sour',
                    'American-Stout','American-Style-Wheat-Wine-Ale','American-Wheat','Baltic-Style-Porter','Barrel-Aged-Beer','Belgian-Style-Blonde-Ale',
                    'Belgian-Style-Dubbel','Belgian-Style-Flanders','Belgian-Style-Fruit-Lambic','Belgian-Style-Golden-Strong-Ale','belgian-style-lambic-gueuze',
                    'Belgian-Style-Pale-Ale','Belgian-Style-Quadrupel','Belgian-Style-Saison','Belgian-Style-Tripel','belgian-style-wit','Berliner-Style-Weisse',
                    'Blonde-Ale','Bohemian-Style-Pilsener','British-Style-Barley-Wine-Ale','California-Common','Chocolate-Beer','Coffee-Beer','Contemporary-Gose',
                    'English-Style-Bitter','English-Style-Brown-Ale','English-Style-Brown-Porter','english-style-india-pale-ale','english-style-dark-mild','English-Style-Oatmeal-Stout',
                    'English-Style-Old-Ale','English-Style-Pale-Ale-(ESB)','English-Style-Sweet-Stout-(Milk-Stout)','European-Style-Export','biere-de-garde',
                    'fruit-field-or-flavored','german-style-brownaltbier','German-Style-Bock','German-Style-Doppelbock','German-Style-Dunkel','German-Style-Dunkelweizen',
                    'German-Style-Hefeweizen','German-Style-Helles','German-Style-Kolsch','german-style-heller-bock-maibock', 'German-Style-Pilsener','German-Style-Schwarzbier','German-Style-Weizenbock','gluten-free','Herb-and-Spice-Beer','Honey-Beer',
                    'Imperial-India-Pale-Ale', 'Irish-Style-Dry-Stout','irish-style-red','New-England-IPA','Pumpkin-Beer','Robust-Porter','Rye-Beer',
                    'scotch-ale-wee-heavy','Scottish-Style-Ale','Session-Beer','Smoke-Beer','Smoke-Porter','Specialty-Beer','Vienna-Style-Lager','german-style-marzen-oktoberfest'] 

list_dict_beers_data=[]
link= "https://www.craftbeer.com/styles/"



for beer in beers_to_scrape2:
    beer_page = f"{link}{beer}" #This is creating the link to the page based on the beers to scrape a for the list beers_to_scrape
    home_response = requests.get(beer_page)
    soup_home = BeautifulSoup(home_response.content, "html.parser") #this is creating the soup for all beers
    color_biterness_alcohol=soup_home.select(".slider-container label a span") #Creates a list with all three characteristics
    
            #Defining the dict inside the loop so it creates a new one on every loop.
   
    dict_beers_data={"beer_style": None,"color": None, "bitterness": None, "alcohol":None}
    #getting the info from the list 
    dict_beers_data["beer_style"]=beer
    dict_beers_data["color"]=color_biterness_alcohol[0].text[0:5]
    dict_beers_data["bitterness"]=color_biterness_alcohol[1].text[0:7]
    dict_beers_data["alcohol"]=color_biterness_alcohol[2].text[0:7]
    list_dict_beers_data.append(dict_beers_data)
    
    time.sleep(3)#resting 3 sec in between beers 
# print(list_dict_beers_data)   

Checking that I got all 80 beers info

In [47]:
len(list_dict_beers_data)


80

Creating the DF and saving it in the raw folder.

In [76]:
df_beers_data = pd.DataFrame(list_dict_beers_data)
df_beers_data.to_csv('../Data/Raw/standard_beer_types_features.csv',index=False)

In [77]:
df_beers_data['beer_style'].nunique()

80

Cleaning the file

In [78]:
df_beers_data=df_beers_data.drop_duplicates().reset_index(drop=True)
df_beers_data['color']=df_beers_data['color'].replace('Varie','N/A')
df_beers_data['bitterness']=df_beers_data['bitterness'].replace('Varies-','N/A')
df_beers_data['alcohol']=df_beers_data['alcohol'].replace('Varies-','N/A')

Obtaining only the numbers using reggex and converting them into float.

In [79]:
color=df_beers_data['color'].str.extract(r'(\d+)-(\d+)').astype(float)
bitterness=df_beers_data['bitterness'].str.extract(r'(\d+)-(\d+)').astype(float)
alcohol=df_beers_data['alcohol'].str.extract(r'(\d+)-(\d+)').astype(float)

Creating bins (categories) as the numbers vary and are not an specific one. The other option was to keep the mean but this seems easier to understand

In [81]:
color_bins = [-np.inf, 10, 20, 30, np.inf]
color_labels = ['Gold', 'Light brown', 'Light dark', 'Dark']
df_beers_data['color'] = pd.cut(df_beers_data['color'], bins=color_bins, labels=color_labels, right=True)

bitterness_bins = [-np.inf, 19, 50, np.inf]
bitterness_labels = ['Low', 'Moderate', 'High']

df_beers_data['bitterness'] = pd.cut(df_beers_data['bitterness'], bins=bitterness_bins, labels=bitterness_labels, right=True)

alcohol_bins = [-np.inf, 4.1, 6.6, np.inf]
alcohol_labels = ['Light', 'Regular', 'Strong']

df_beers_data['alcohol'] = pd.cut(df_beers_data['alcohol'], bins=alcohol_bins, labels=alcohol_labels, right=True)

In [83]:
df_beers_data=df_beers_data.dropna()

Creating the id for the styles that I will use to link with the other DS

In [85]:
df_beers_data['style_id']=np.arange(1, 73)


In [108]:
df_beers_data['beer_style']=df_beers_data['beer_style'].str.replace("-"," ")

In [118]:
df_beers_data

,beer_style,color,bitterness,alcohol,style_id
0,Amber Ale,Light brown,Moderate,Regular,1
1,American Amber Lager,Gold,Moderate,Regular,2
2,American Barley Wine,Light brown,High,Strong,3
3,American Black Ale,Dark,High,Regular,4
5,American Brown Ale,Light dark,Moderate,Light,5
...,...,...,...,...,...
73,Scottish Style Ale,Light brown,Low,Regular,68
74,Session Beer,Dark,Moderate,Regular,69
76,Smoke Porter,Light brown,Moderate,Regular,70
78,Vienna Style Lager,Light brown,Moderate,Regular,71


In [ ]:
# df_beers_data.to_csv('../Data/Clean/standard_beer_types_features_clean.csv',index=False)